In [1]:
import numpy as np
import pandas as pd
import os
import tqdm
import itertools
from collections import Counter

import sys
sys.path.append('../')

from util.text_preprocessing import tokenize_okt

# Load Data

## 전체 코멘트 불러오기

In [2]:
total_comments_df = pd.read_pickle("dataset/total_raw_token/0710 comments_with_okt-tokenized.pkl")
total_comments_df = total_comments_df.loc[:,['comment_id','comment','comment_token']]
total_comments_df.head(5)

,comment_id,comment,comment_token
0,F3C2DFCFE9E947B19C8A3053DD5C821B,이국종교수는 외과 야전사령관 이다. 그분의 업무에 차질 없도록 물심양면 으로 ...,"[이국, 종, 교수, 외과, 야전, 사령관, 분, 업무, 차질, 없다, 물, 심양,..."
1,61A3B6AAD6124DFFB70EC4275496F8A6,"문재인 대통령님과 이국종 교수님, 그리고 귀순병사 모두 건강하시길 빕니다.","[문재인, 대통령, 이국, 종, 교수, 귀순, 병사, 모두, 건강하다, 비다]"
2,B6660EF2C7C14A2F903A08363641CF09,단 한번도 써보지 못했다ㆍ 포인트의 절반만이라도 통신비로 차감을!,"[단, 한번, 써다, 보지, 포인트, 절반, 통신비, 차감]"
3,C4F129BFB1FD4543A24A03D96F38351D,우리나라 통신업체들 참 쉽게 돈 벌어요,"[우리나라, 통신, 업체, 차다, 쉬다, 돈, 벌다]"
4,1A21FDDF151E45DF808F0E5B662E8F3C,진짜 쓸데가너무적다.,"[진짜, 쓸다, 너무, 적다]"


## 감정 사전 만들기
- 6 emotion : happy, sad, disgust, angry, surprised, fear
- 홍종선, 정연주. (2009). 감정동사의 범주 규정과 유형 분류. 한국어학, 45(), 387-420.

In [3]:
six_emodict_df = pd.read_excel("자료/six_emotion_bigram2_0403.xlsx", encoding = 'utf-8')
six_emodict_df.head()

,happy,sad,disgust,angry,surprised,fear
0,가뿐하다,가슴앓이,가소롭다,갈기갈기,갑작스럽다,가혹하다
1,감개무량하다,가엽다,거북하다,개새끼,경악하다,강압적
2,감격스럽다,가엾다,경박하다,개자식,경이,겁쟁이
3,감격하다,가혹하다,괴상하다,격노하다,급작스럽다,공포감
4,감동스럽다,각박하다,괴팍하다,격분하다,기겁하다,공포스럽다


In [4]:
# 감정 별 단어 개수
six_emodict_df.count()

happy        113
sad          174
disgust       77
angry        110
surprised     47
fear          38
dtype: int64

In [5]:
# 감정 리스트에 변수 할당 
six_emotion = six_emodict_df.columns.tolist()
print(six_emotion)
for emo in six_emotion:
    # 단어 사전에서 emo_words 받아오기
    emo_words = six_emodict_df[emo].dropna().tolist() 
    # 단어 토큰화, 토큰화 후 nan값 된 단어 제거
    emo_words = [tokenize_okt(x)[0] for x in emo_words if len(tokenize_okt(x))>0]
    # 중복 제거
    emo_words = sorted(list(set(emo_words)))
    # 각 감정변수에 차례대로 할당
    globals()[emo] = emo_words
    
angry[:5]

['happy', 'sad', 'disgust', 'angry', 'surprised', 'fear']


['가당찮다', '가증', '갈기갈기', '갈등', '강압']

In [6]:
bi_happy = [('속','시원하다'),('낯','간지럽다'),('정','들다'),('홀','가분')]

bi_sad = [('가슴','아프다'),('기운','없다'),('남','부럽다'),('뒤','숭숭'),('맥','빠지다'),
              ('복','받치다'),('뼈','아프다'),('뼈','저리다'),('애','끓다'),('애','닮다'),
              ('애','타다'),('처','연하다'),('속','썩다'),('속','앓이'),('속','타다')]

bi_disgust =  [('못','밉다'),('정','떨어지다'),('못','마땅하다'),('시','덥다'),('남','부끄럽다'),
                   ('징','그렇다'),('뼈','저리다'),('애','끓다'),('애','닮다'),('처','연하다'),
                   ('속','썩다'),('속','앓이'),('속','타다')]

bi_angry = [('못','되다'),('삐','딱하다'),('비','딱하다'),('약','오르다'),('치','떨리다'),
                ('치밀','오르다'),('성','나다'),('못','마땅하다'),('정','떨어지다'),('남','부끄럽다')]

bi_surprised = [('기상','천외'),('얼','떨다')]

bi_fear = [('뒤','숭숭')]

# 코멘트 데이터에 감정 레이블 달기 

In [7]:
test = ["가슴 아프다 정말", #- sad
        "요즘 상황이 뒤숭숭하다",#sad fear
        "어어어어엉 우웅우 감동스럽다",#happy
       "낯간지러워"]
        
test_token = []
for i in test:
    test_token.append(tokenize_okt(i))
print(test_token)

[['가슴', '아프다', '정말'], ['요즘', '상황', '뒤', '숭숭'], ['어어', '엉', '우웅', '우', '감동', '스럽다'], ['낯', '간지럽다']]


## 댓글에 특정 감정에 해당하는 단어가 등장하면 update

- bigram
- unigram

In [8]:
total_comments_df['comment_token'][:10]

0    [이국, 종, 교수, 외과, 야전, 사령관, 분, 업무, 차질, 없다, 물, 심양,...
1          [문재인, 대통령, 이국, 종, 교수, 귀순, 병사, 모두, 건강하다, 비다]
2                    [단, 한번, 써다, 보지, 포인트, 절반, 통신비, 차감]
3                        [우리나라, 통신, 업체, 차다, 쉬다, 돈, 벌다]
4                                     [진짜, 쓸다, 너무, 적다]
5                          [포인트, 쓰다, 내, 돈, 더, 많이, 나가다]
6                               [사법, 적폐, 청산, 시급하다, 싶다]
7    [죽다, 자하, 산다, 죽음, 사선, 넘어오다, 그대, 대한민국, 에세, 아름답다,...
8    [당시, 우리, 국군, 미군, 대응, 아주, 보이다, 공동, 경비, 구역, 모든, ...
9                             [우리, 국방부, 대응, 문제, 언론, 임]
Name: comment_token, dtype: object

In [9]:
emo_set = [happy, sad, disgust, angry, surprised, fear]
emo_bigram_set = [bi_happy, bi_sad, bi_disgust, bi_angry, bi_surprised, bi_fear]

check_token_lst = []
emo_cnt_lst = []

total_num = len(total_comments_df)

# comment 돌기
for i,comment in enumerate(total_comments_df['comment_token']):
    print(i/total_num,end="\r")
    check_token = []
    emo_cnt = np.zeros(len(emo_set))
    
    # bigram 처리
    for j, biwords in enumerate(emo_bigram_set):
        for a, b in biwords:
             if (a in comment) & (b in comment):
                check_token.append(a+b)
                emo_cnt[j] += 1 

    # unigram 처리
    for token in comment:
        for k, words in enumerate(emo_set):
            if token in words:
                emo_cnt[k] += 1
                check_token.append(token)

    check_token_lst.append(check_token)
    emo_cnt_lst.append(emo_cnt)

In [10]:
# 비율로 환산
sum_of_emo = np.array(emo_cnt_lst).sum(axis=1,keepdims=True)

emo_ratio = np.array(emo_cnt_lst) / sum_of_emo
emo_ratio[np.isnan(emo_ratio)] = 0

emo_df = pd.DataFrame(emo_ratio,columns = ['happy','sad','disgust','angry','surprised','fear'])

/Users/kim-woojeong/miniconda/envs/python_workspace/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [11]:
comment_with_emo = pd.concat([emo_df,
                              total_comments_df,
                              pd.DataFrame({"check":pd.Series(check_token_lst)})],
                             sort=False,axis=1)
# 최소 1개의 감정단어 등장한 댓글만 뽑기
comment_with_emo = comment_with_emo[comment_with_emo['check'].apply(len) > 0]
print(comment_with_emo.shape)
comment_with_emo.head()

(489774, 10)


,happy,sad,disgust,angry,surprised,fear,comment_id,comment,comment_token,check
6,0.0,0.0,0.0,1.0,0.0,0.0,0DC01BDF77744B58957E7A39F3E98A7B,사법 적폐 청산이 시급한듯 싶네요!,"[사법, 적폐, 청산, 시급하다, 싶다]",[적폐]
17,0.0,0.0,0.0,1.0,0.0,0.0,194B2B0BB3A642F1AD0670EED8BFCE46,"공동경비구역에서 교전이 벌어지면 뭐가 좋을까? 개무리, 자위당 놈들 비양거리는 소리...","[경비, 구역, 교전, 벌어지다, 뭐, 좋다, 개, 무리, 자위, 놈, 비양, 거리...",[놈]
21,0.0,0.0,0.0,0.0,0.0,1.0,22C76F3161F242A59AF956299EBC41EB,연평도 K9은 북한의 포 공격이 끝나고 난 후에 대응했다. 그것도 잘못인가? 전방부...,"[연평도, 은, 북한, 포, 공격, 끝나다, 난, 후, 대응, 그것, 잘못, 방부,...",[공포]
34,0.0,0.0,0.5,0.5,0.0,0.0,87EFAC2FA51D4A25B73CBE30313E22A8,언제나 북한에 당하고 끌려다니는 남한. 남한을 향하여 총질을 해도 지켜보기나 하고...,"[언제나, 북한, 당하다, 끌리다, 다니다, 남한, 남한, 향, 질, 해도, 지켜보...","[한심하다, 한심하다]"
41,0.0,0.0,0.0,0.0,0.0,1.0,87063866023E432AB27F17696EA988A1,"양측이 첨예하게 대치하고 있는 상황에서...수십발의 총성이 울리고,남측까지 추격전...","[양, 첨예하다, 대치, 있다, 상황, 발의, 총성, 울리다, 남, 추격, 전이, ...",[두렵다]


In [12]:
print("{}개의 댓글 중 {} 개의 댓글이 감정 단어를 보유.".format(len(total_comments_df),len(comment_with_emo)))

1780402개의 댓글 중 489774 개의 댓글이 감정 단어를 보유.


In [13]:
comment_with_emo['check'].apply(len).max()

109

In [14]:
comment_with_emo[comment_with_emo['check'].apply(len)==109]

,happy,sad,disgust,angry,surprised,fear,comment_id,comment,comment_token,check
210699,0.0,0.963303,0.0,0.036697,0.0,0.0,6572A94D88CA4BF19206E402F4787649,흑흑흑흑 흑흑흑흑 흑흑흑흑 흑흑흑흑 흑흑흑흑 흑흑흑흑 흑흑흑흑 흑흑흑흑 흑흑흑흑 흑...,"[흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, ...","[흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, 흑, ..."


## 감정 단어 보유 비율에 따라 labeling

In [15]:
def label_with_major_emotion(df, ratio):
    over_index = np.where((df.happy > ratio) |
                          (df.sad > ratio) |
                          (df.disgust > ratio) |
                          (df.angry > ratio) |
                          (df.surprised > ratio) |
                          (df.fear > ratio))[0]

    over_df = df.iloc[over_index.tolist(),:]

    # Skipna = True will skip all the Na values 
    # find maximum along column axis 
    over_df['emotion'] = over_df.iloc[:,:6].idxmax(axis = 1, skipna = True)
    print(Counter(over_df.emotion))
    
    return over_df

In [16]:
label_with_major_emotion(comment_with_emo, 0.5).to_pickle("dataset/model/0710 comment_with_emo_over0.5.pkl")

/Users/kim-woojeong/miniconda/envs/python_workspace/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


Counter({'angry': 240971, 'sad': 83041, 'happy': 44682, 'disgust': 17263, 'fear': 14649, 'surprised': 8001})
